In [1]:
import sys
import os
lib = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)



In [2]:
import os
from os.path import join
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
sklearn.__version__
%matplotlib notebook

In [3]:
def get_informative_df(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    data = pickle.load(open(sample_path, 'rb'))
    interesting_properties = ['is_apoptosis', 'is_immune', 'is_cancer', 'cancer_immune_conflict', 'is_doublet']
    relevant_cells = {key:[cf for cf in data.cells_information if getattr(cf, key)] for key in interesting_properties}
    n_cells = len(relevant_cells)
    counter = {key1:{key2: 0 for key2 in interesting_properties} for key1 in interesting_properties}
    for prop1 in interesting_properties:
        for prop2 in interesting_properties:
            for cell_inf in relevant_cells[prop1]:
                counter[prop1][prop2] += int(getattr(cell_inf, prop2))
    df = pd.DataFrame(counter.values(), index=counter.keys())[counter.keys()]
    return  df

def check_cross_cell_types(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    types = ['None', 'Immune_general', 'T cells', 'CD4 helper T cells', 'CD8 Cytotoxic T cells', 'Regulatory T cells',
             'Regulatory CD4 T cells', 'Regulatory CD8 T cells', 'Regulatory CD4_CD8 T cells', 'NKT cells', 'NK cells',
             'B cells', 'Activated T cells', 'Senescence T cells', 'Terminal effector', 'Exhausted T cells',
             'Stem_like T cells', 'Memory T cells', 'Memory CD4 T cells', 'Memory CD8 T cells',
             'Memory CD4_CD8 T cells', 'Macrophage_immature', 'Macrophage_mature', 'Monocyte_immature',
             'Monocyte_mature', 'cDCs_dendritic_cells', 'pDCs', 'myeloid cells_general_immature',
             'myeloid cells_general_mature', 'Neutrophils', 'Granolocytes']
    table = {k1: {k2: 0 for k2 in types} for k1 in types}
    data = pickle.load(open(sample_path, 'rb'))
    cls = [list(set(v)) for v in data.cells_information.getattr('cell_type_list')]
    cls = [v if len(v) else ['None'] for v in cls]

    for cell in cls:
        for t1 in cell:
            for t2 in cell:
                table[t1][t2] += 1
    df = pd.DataFrame(table.values(), index=table.keys())[table.keys()]
    return df

# Investigate properties of selected sample
Specify sample name and samples path

In [149]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
sample_id = 'M108'

In [150]:
get_informative_df(sample_id)

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet
is_apoptosis,21167,37,15272,121,4472
is_immune,37,116,0,0,10
is_cancer,15272,0,17416,0,3924
cancer_immune_conflict,121,0,0,213,38
is_doublet,4472,10,3924,38,4957


# Investigate properties over all samples
Specify samples path and samples you're not interested in investigating them.

In [57]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
not_investigate = ['M133', 'M108']


In [58]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
all_df = None
for sample in samples:
#     print(sample)
    df = get_informative_df(sample)
    if all_df is None:
        all_df = df
    else:
        all_df = all_df.add(df)

all_df

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet
is_apoptosis,27729,1680,11787,1391,207
is_immune,1680,91449,0,0,1164
is_cancer,11787,0,96024,0,173
cancer_immune_conflict,1391,0,0,31994,959
is_doublet,207,1164,173,959,2505


In [ ]:
# If you want to save result
OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_properties.xlsx'
all_df.to_excel(OUTPATH)

# Cross-checking cell-types of selected sample

In [48]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
sample_id = 'M108'

In [52]:
check_cross_cell_types(sample_id).head()

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,7314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,2946,1398,135,705,30,6,0,0,57,...,3,24,3,87,120,0,0,135,651,159
T cells,0,1398,1398,135,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD4 helper T cells,0,135,135,135,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,705,705,15,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Cross-checking cell-types over all samples

In [4]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
not_investigate = [] #['M133', 'M108']

In [62]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
all_df = None
for sample in samples:
#     print(sample)
    df = check_cross_cell_types(sample)
    if all_df is None:
        all_df = df
    else:
        all_df = all_df.add(df)

all_df

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,206367,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,85127,48882,5995,25287,817,508,78,49,2273,...,96,2880,167,4661,3009,88,60,4116,12813,953
T cells,0,48882,48882,5995,25287,2,1,0,0,371,...,0,0,10,0,0,0,0,0,0,0
CD4 helper T cells,0,5995,5995,5995,377,0,1,0,0,41,...,0,0,5,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,25287,25287,377,25287,0,0,0,0,89,...,0,0,3,0,0,0,0,0,0,0
Regulatory T cells,0,817,2,0,0,817,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4 T cells,0,508,1,1,0,0,508,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD8 T cells,0,78,0,0,0,0,0,78,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4_CD8 T cells,0,49,0,0,0,0,0,0,49,0,...,0,0,0,0,0,0,0,0,0,0
NKT cells,0,2273,371,41,89,1,0,0,0,2273,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# If you want to save result
OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_cell_types_2.xlsx'
all_df.to_excel(OUTPATH)

# DEBUG

In [21]:
path = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\classifying_cell_types\5.12.20'
sample_id = r'M132'
sample_path = join(path, sample_id, f'{sample_id}.pkl')
data = pickle.load(open(sample_path, 'rb'))

In [7]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
all_df = None
count = 0
for sample in samples:
    print(sample)
    sample_path = join(SAMPLES_PATH, sample, f'{sample}.pkl')
    data = pickle.load(open(sample_path, 'rb'))
    xx = sum([c.is_immune for c in data.cells_information])
    print(xx)
    count += xx
    
    yy = [1 for c in data.cells_information if c.is_immune == False and  len(c.cell_type_list)>0]
    print(sum(yy))
print(count)

M100
190
0
M101
1467
0
M102
734
0
M103
433
0
M104
277
0
M105
3094
0
M106
562
0
M107
1627
0
M108
116
0
M109
2121
0
M110
1869
0
M111
5658
0
M112
178
0
M114
4530
0
M115
2910
0
M116
3380
0
M118
1023
0
M120
1219
0
M121
4071
0
M122
676
0
M123
3368
0
M124
4274
0
M125
140
0
M126
2757
0
M127
78
0
M128
422
0
M129
527
0
M130
3273
0
M131
3986
0
M132
2663
0
M133
83
0
M134
1798
0
M135
1955
0
M136
1268
0
M137
3758
0
M138
2428
0
M139
2428
0
M140
3395
0
M141
85
0
M143
1168
0
M144
538
0
M145
3983
0
M146
5783
0
M97
1299
0
M98
760
0
M99
3296
0
91648
